In [1]:
import pickle, tqdm, time, os, tensorflow_hub as hub, numpy as np
from pymongo import MongoClient
from pprint import pprint

In [2]:
def getMongoUri(auth_filename='auths/mongo_uri.pkl'):
    with open(auth_filename, 'rb') as file:
        pwd = pickle.load(file)
    return pwd['mongo_uri']

def createMongoClient(db='aita'):
    mongo_uri = getMongoUri()
    client = MongoClient(mongo_uri)
    db = client.aita
    return client, db

In [3]:
client, db = createMongoClient()
collection = db.posts

**End Goal - Given a query input, find most similar document.**

Potential Solution 1:

1. Input Query --> Embedding
2. Embedding X Embedding Matrix = Similarity Scores
3. Similarity Scores Sort Descending (keep id column)
4. Get Top N actual posts & titles by using ID


In [4]:
NUM_DOCS = collection.count_documents({})

In [5]:
# load & cache tensorflow model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
embed(['testing'])
print('model cached')

model cached


**Converting Each Post From Text To Embedding**
- Start off with using the title of the post only

In [8]:
# fetch & store titles,id for each post in aita.
# titles will be used in batches later for conversion to embeddings & id is needed for lookups
# TODO: Optimize - dont need all titles stored locally, embed and load titles in same loop
ids, titles = [], []
for post in tqdm.tqdm_notebook(collection.find(projection={'title': 1, 'id': 1}), total=NUM_DOCS):
    title = post['title']
    post_id = post['id']
    ids.append(post_id)
    titles.append(title)

/Users/alamjeetsingh/opt/anaconda3/envs/cs448/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/682771 [00:00<?, ?it/s]

In [9]:
def calcEndIdx(start_idx, batch_size, ndocs):
    end_idx = start_idx + batch_size
    end_idx = ndocs if end_idx > ndocs - 1 else end_idx
    return end_idx

In [31]:
# convert text to embeddings in batches (model can handle multiple texts at once)
# batch size depends on compute power
embeddings = None
BATCH_SIZE = 1000
for start_idx in tqdm.tqdm_notebook(range(0, NUM_DOCS, BATCH_SIZE)):
    end_idx = calcEndIdx(start_idx, BATCH_SIZE, NUM_DOCS)
    curr_embeddings = embed(titles[start_idx:end_idx]).numpy()
    if embeddings is None:
        embeddings = curr_embeddings
    else:
        embeddings = np.concatenate((embeddings, curr_embeddings))
    
    if start_idx == 1000:
        break

/Users/alamjeetsingh/opt/anaconda3/envs/cs448/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/683 [00:00<?, ?it/s]